# 🚀 Capa Silver: Procesamiento y Estructuración de Pedidos E-commerce

Este notebook describe el proceso de transformación de datos desde una fuente semi-estructurada (**JSON**) hacia una tabla optimizada en formato **Delta** (Capa Silver). 

### 🛠️ Objetivo Técnico
La tarea principal es el **aplanamiento (flattening)** de la entidad de pedidos. Dado que la información detallada de cada compra reside en la estructura anidada `line_items`, aplicamos lógica de **PySpark** para desglosar cada artículo en una fila independiente.

### 📈 Impacto en el Negocio
Al estructurar los datos de esta manera, habilitamos:
* **Reportes de Ventas:** Análisis de ingresos por producto y categoría.
* **Customer Intelligence:** Identificación de patrones de compra y frecuencia de clientes.
* **Preparación para ML:** Datos listos para alimentar modelos de recomendación.

In [0]:
dbutils.widgets.text("input_path", "/Volumes/woocommerce/woocommerce_db/json_sources/", "Ruta de Origen")
input_path = dbutils.widgets.get("input_path")

In [0]:
#importar librerías que se utilizarán en el notebook
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.types import StructType, StructField, StringType, LongType, TimestampType,ArrayType


In [0]:
%sql
create schema if not exists woocommerce;
create database if not exists woocommerce_db;
create volume if not exists woocommerce.woocommerce_db.json_sources

### En este momento hay que cargar orders_final.json al volume creado en el paso anterior, el archivo debe aparecer al ejecutar la siguiente linea

In [0]:
display(dbutils.fs.ls(input_path))

path,name,size,modificationTime
dbfs:/Volumes/woocommerce/woocommerce_db/json_sources/orders_final.json,orders_final.json,42087112,1769952681000


In [0]:
# 1. Definición del Esquema (Garantiza robustez en la ingesta)
schema = StructType([
    StructField("id", LongType(), True),                     # ID de la Orden
    StructField("status", StringType(), True),               # Para filtrar solo ventas 'completed'
    StructField("customer_id", LongType(), True),            # CLAVE para Modelos de Recomendación
    StructField("date_created", TimestampType(), True),      # Base para comparaciones mensuales
    StructField("date_modified", TimestampType(), True),     # fecha de actualizacion de ordenes
    StructField("total", StringType(), True),                # Ingreso total de la orden
    StructField("discount_total", StringType(), True),       # Para medir impacto de promociones
    StructField("payment_method", StringType(), True),       # Análisis de métodos de pago
    StructField("shipping_total", StringType(), True),
    # Billing: 
    StructField("billing", StructType([
        StructField("first_name", StringType(), True),
        StructField("last_name", StringType(), True),
        StructField("email", StringType(), True),
        StructField("state", StringType(), True),            # Reporte por región/departamento
        StructField("phone", StringType(), True)
    ])),
    
    # Line Items:
    StructField("line_items", ArrayType(StructType([
        StructField("product_id", LongType(), True),         # Item ID para recomendación
        StructField("name", StringType(), True),             # Nombre del producto
        StructField("quantity", LongType(), True),           # Volumen de venta
        StructField("total", StringType(), True),            # Ingreso por producto individual
        StructField("subtotal", StringType(), True),         # Ingreso por producto individual
        StructField("subtotal_tax", StringType(), True)      # impuestos

    ])))
])
# 2. Lectura de la fuente JSON (Capa Bronze)
# Nota: Se asume que los archivos están depositados en el volumen de Databricks
orders_df = spark.read.schema(schema).option("header",True).json(f"{input_path}/*.json")

display(orders_df.limit(20))

id,status,customer_id,date_created,date_modified,total,discount_total,payment_method,shipping_total,billing,line_items
1,processing,0,2026-01-30T00:00:00.000Z,2026-01-30T00:00:00.000Z,109,0,woo-mercado-pago-custom,8,"List(nombre_1, apellido_1, email_1@test.com, LIM, telefono_1)","List(List(1, producto_1, 1, 84, 84, 15))"
2,processing,0,2026-01-29T00:00:00.000Z,2026-01-29T00:00:00.000Z,125,0,woo-mercado-pago-custom,8,"List(nombre_2, apellido_2, email_2@test.com, LIM, telefono_2)","List(List(2, producto_2, 1, 97, 97, 17))"
3,processing,0,2026-01-29T00:00:00.000Z,2026-01-29T00:00:00.000Z,317,0,bacs,8,"List(nombre_3, apellido_3, email_3@test.com, LIM, telefono_3)","List(List(3, producto_3, 1, 109, 109, 19), List(4, producto_4, 1, 150, 150, 27))"
4,processing,0,2026-01-29T00:00:00.000Z,2026-01-29T00:00:00.000Z,106,0,woo-mercado-pago-custom,10,"List(nombre_4, apellido_4, email_4@test.com, LIM, telefono_4)","List(List(5, producto_5, 1, 79, 79, 14))"
5,processing,595,2026-01-29T00:00:00.000Z,2026-01-29T00:00:00.000Z,348,0,woo-mercado-pago-custom,8,"List(nombre_5, apellido_5, email_5@test.com, LIM, telefono_5)","List(List(6, producto_6, 2, 286, 286, 51))"
6,processing,0,2026-01-28T00:00:00.000Z,2026-01-28T00:00:00.000Z,400,0,bacs,8,"List(nombre_6, apellido_6, email_6@test.com, LIM, telefono_6)","List(List(7, producto_7, 1, 148, 148, 26), List(8, producto_8, 1, 182, 182, 32), List(9, producto_9, 1, 0, 0, 0))"
7,processing,0,2026-01-28T00:00:00.000Z,2026-01-28T00:00:00.000Z,386,0,woo-mercado-pago-custom,8,"List(nombre_7, apellido_7, email_7@test.com, LIM, telefono_7)","List(List(10, producto_10, 1, 159, 159, 28), List(11, producto_11, 1, 109, 109, 19), List(12, producto_12, 1, 50, 50, 9), List(9, producto_9, 1, 0, 0, 0))"
8,processing,0,2026-01-28T00:00:00.000Z,2026-01-28T00:00:00.000Z,153,0,woo-mercado-pago-custom,11,"List(nombre_8, apellido_8, email_8@test.com, LIM, telefono_8)","List(List(13, producto_13, 1, 118, 118, 21))"
9,on-hold,0,2026-01-28T00:00:00.000Z,2026-01-28T00:00:00.000Z,179,0,bacs,8,"List(nombre_9, apellido_9, email_9@test.com, LIM, telefono_9)","List(List(14, producto_14, 1, 143, 143, 25))"
10,processing,0,2026-01-27T00:00:00.000Z,2026-01-27T00:00:00.000Z,364,0,cod,8,"List(nombre_10, apellido_10, email_10@test.com, LIM, telefono_10)","List(List(15, producto_15, 1, 131, 131, 23), List(16, producto_16, 1, 168, 168, 30))"


In [0]:
from pyspark.sql.functions import col, explode

# 1. "Explotamos" el arreglo de line_items. 
# Esto genera una fila por cada producto vendido, manteniendo los datos de la orden.
df_exploded = orders_df.withColumn("item", explode(col("line_items")))

# 2. Seleccionamos y aplanamos basándonos en el esquema enriquecido.
# Accedemos a los campos anidados usando el punto (ej. billing.city)
df_final = df_exploded.select(
    col("id").alias("order_id"),
    col("customer_id"),             # Crucial para identificar mejores clientes y ML
    col("status"),
    col("date_created"),
    col("date_modified"),
    col("discount_total"),
    col("payment_method"),
    col("total").cast("double"),    # Para KPIs de ingresos
    col("shipping_total").cast("double"),
    
    # Aplanando Billing (Geografía para reportes de ventas)
    col("billing.first_name").alias("customer_name"),
    col("billing.last_name").alias("customer_last_name"),
    col("billing.email").alias("customer_email"),
    col("billing.phone").alias("customer_phone"),
    col("billing.state").alias("customer_state"),
    
    # Aplanando Line Items (Directo del objeto 'item' explotado)
    col("item.product_id").alias("product_id"),
    col("item.name").alias("product_name"),
    col("item.quantity").alias("quantity"),
    col("item.subtotal").cast("double").alias("item_subtotal"),
    col("item.total").cast("double").alias("item_total"),
    col("item.subtotal_tax").cast("double").alias("item_subtotal_tax")
)

# 3. Guardar como Tabla Delta
# Esto es lo que define tu capa Silver profesional
df_final.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("woocommerce.woocommerce_db.silver_orders")

# Verificamos el resultado final
display(spark.read.table("woocommerce.woocommerce_db.silver_orders").limit(20))

order_id,customer_id,status,date_created,date_modified,discount_total,payment_method,total,shipping_total,customer_name,customer_last_name,customer_email,customer_phone,customer_state,product_id,product_name,quantity,item_subtotal,item_total,item_subtotal_tax
1,0,processing,2026-01-30T00:00:00.000Z,2026-01-30T00:00:00.000Z,0,woo-mercado-pago-custom,109.0,8.0,nombre_1,apellido_1,email_1@test.com,telefono_1,LIM,1,producto_1,1,84.0,84.0,15.0
2,0,processing,2026-01-29T00:00:00.000Z,2026-01-29T00:00:00.000Z,0,woo-mercado-pago-custom,125.0,8.0,nombre_2,apellido_2,email_2@test.com,telefono_2,LIM,2,producto_2,1,97.0,97.0,17.0
3,0,processing,2026-01-29T00:00:00.000Z,2026-01-29T00:00:00.000Z,0,bacs,317.0,8.0,nombre_3,apellido_3,email_3@test.com,telefono_3,LIM,3,producto_3,1,109.0,109.0,19.0
3,0,processing,2026-01-29T00:00:00.000Z,2026-01-29T00:00:00.000Z,0,bacs,317.0,8.0,nombre_3,apellido_3,email_3@test.com,telefono_3,LIM,4,producto_4,1,150.0,150.0,27.0
4,0,processing,2026-01-29T00:00:00.000Z,2026-01-29T00:00:00.000Z,0,woo-mercado-pago-custom,106.0,10.0,nombre_4,apellido_4,email_4@test.com,telefono_4,LIM,5,producto_5,1,79.0,79.0,14.0
5,595,processing,2026-01-29T00:00:00.000Z,2026-01-29T00:00:00.000Z,0,woo-mercado-pago-custom,348.0,8.0,nombre_5,apellido_5,email_5@test.com,telefono_5,LIM,6,producto_6,2,286.0,286.0,51.0
6,0,processing,2026-01-28T00:00:00.000Z,2026-01-28T00:00:00.000Z,0,bacs,400.0,8.0,nombre_6,apellido_6,email_6@test.com,telefono_6,LIM,7,producto_7,1,148.0,148.0,26.0
6,0,processing,2026-01-28T00:00:00.000Z,2026-01-28T00:00:00.000Z,0,bacs,400.0,8.0,nombre_6,apellido_6,email_6@test.com,telefono_6,LIM,8,producto_8,1,182.0,182.0,32.0
6,0,processing,2026-01-28T00:00:00.000Z,2026-01-28T00:00:00.000Z,0,bacs,400.0,8.0,nombre_6,apellido_6,email_6@test.com,telefono_6,LIM,9,producto_9,1,0.0,0.0,0.0
7,0,processing,2026-01-28T00:00:00.000Z,2026-01-28T00:00:00.000Z,0,woo-mercado-pago-custom,386.0,8.0,nombre_7,apellido_7,email_7@test.com,telefono_7,LIM,10,producto_10,1,159.0,159.0,28.0


### Haremos una consulta simple para comenzar con nuestra analitica, el top 5 de los productos mas vendidos.

In [0]:
%sql
-- Visualización rápida: Top 5 productos con mayor volumen de ventas
SELECT product_name, SUM(quantity) as total_vendido
FROM woocommerce.woocommerce_db.silver_orders
GROUP BY product_name
ORDER BY total_vendido DESC
LIMIT 5